In [2]:
from clip_image_to_text import load_interrogator, inference
model = load_interrogator("/home/mlfavorfit/lib/favorfit/kjg/0_model_weights/image_to_text/clip", caption_model_name="blip-base", device="cuda") 

In [4]:
from PIL import Image

img_pil = Image.open("/media/mlfavorfit/sdb/contolnet_dataset/val/158_Chane.jpg").convert('RGB')
result = inference(img_pil=img_pil, model=model)
print(result)

chan chan perfume bottle on a black background, perfume bottle, dramatic product photography, antique perfume, chaumet, floating in perfume, commercial product photography, chanel, dramatic product shot, miniature photography, product photography, rendered with unreal engine, product photography 4 k, 3 d product render, still life photography, commercial photo, packshot, behance. polished


In [5]:
from glob import glob

fns = glob("/media/mlfavorfit/sdb/contolnet_dataset/control_net_train_base/images/*")

In [6]:
result_dict_list = []

In [9]:
import os
from tqdm import tqdm

for fn in tqdm(fns, total=len(fns)):
    temp_dict = {"text":None, "image":None, "mask":None}

    img = Image.open(fn).convert("RGB")
    caption = inference(img_pil=img, model=model)

    temp_dict["text"] = caption
    temp_dict["image"] = os.path.join("images", os.path.basename(fn))
    temp_dict["mask"] = os.path.join("masks", os.path.basename(fn))

    result_dict_list.append(temp_dict)

100%|██████████| 197967/197967 [9:49:27<00:00,  5.60it/s]  


In [10]:
import json
def save_data_to_jsonlines(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            json.dump(item, f)
            f.write('\n')

In [11]:
save_data_to_jsonlines(result_dict_list, './temp.jsonl')

In [20]:
import json

temp = []
with open("/media/mlfavorfit/sdb/contolnet_dataset/inpaint_train/inpaint_train.jsonl", mode="r") as f:
    for cur in f:
        temp.append(json.loads(cur))

In [26]:
resuit_tokens = []

In [27]:
from transformers import CLIPProcessor
import nltk
from nltk.corpus import stopwords

# # NLTK의 불용어 리스트 다운로드
# nltk.download("stopwords",)

# # 영어 불용어 리스트 가져오기
# stopwords_list = stopwords.words("english")

# CLIP tokenizer 로드
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def create_ngrams(tokens, n):
    ngrams = []
    for i in range(len(tokens) - n + 1):
        ngrams.append(" ".join(tokens[i:i+n]))
    return ngrams

# 텍스트를 토큰화할 함수 정의
def tokenize_text(text):
    # 텍스트를 토큰화
    tokens = clip_processor.tokenizer.tokenize(text)
    # # 불용어를 제외한 토큰만 선택하여 반환
    # tokens = [token for token in tokens if token.lower() not in stopwords_list]
    tokens = [cur.replace("</w>", "") for cur in tokens]
    return create_ngrams(tokens, 2)

# 토큰화할 텍스트
text = "A cat is sitting on the mat."

# 텍스트를 토큰화하여 불용어를 제외한 토큰 리스트 얻기
tokens = tokenize_text(text)

# 토큰 출력
print("Tokens:", tokens)


Tokens: ['a cat', 'cat is', 'is sitting', 'sitting on', 'on the', 'the mat', 'mat .']


In [28]:
from tqdm import tqdm

for cur in tqdm(temp, total=len(temp)):
    tokens = tokenize_text(cur["text"])
    resuit_tokens.extend(tokens)

100%|██████████| 197967/197967 [00:25<00:00, 7671.45it/s]


In [29]:
from collections import Counter

counter = Counter(resuit_tokens)

In [41]:
aa = dict(counter).items()

In [44]:
sorted(aa, key=lambda x:x[1], reverse=True)

[('photography ,', 158164),
 ('photo ,', 85843),
 ('food photography', 75952),
 ('on a', 69555),
 (', by', 52236),
 ('inspired by', 48823),
 (', inspired', 48591),
 ('on unsplash', 48025),
 ('background ,', 47299),
 ('with a', 47086),
 ('unsplash ,', 46250),
 (', trending', 46015),
 ('trending on', 45166),
 (', award', 39435),
 (', food', 38286),
 ('of a', 36027),
 ('4 k', 33089),
 ('beh ance', 31416),
 ('lighting ,', 31173),
 ('on the', 30618),
 (', beh', 30423),
 ('table ,', 30360),
 ('award winning', 30000),
 ('ance .', 29714),
 ('. polished', 29640),
 ('polished ,', 28165),
 ('coffee ,', 27979),
 ('product photo', 27619),
 ('in the', 27122),
 (', 4', 26711),
 ('in a', 26643),
 ('winning food', 26435),
 ('featured on', 25502),
 (', photo', 25314),
 (', featured', 23708),
 ('and a', 23216),
 (', the', 21538),
 (', an', 21236),
 ('food photo', 21073),
 (', product', 21007),
 (', 3', 20403),
 ('of coffee', 20013),
 ('a table', 19615),
 (', detailed', 19591),
 (', white', 19113),
 ('an 